<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Discovering-customer-attrition-patterns" data-toc-modified-id="Discovering-customer-attrition-patterns-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Discovering customer attrition patterns</a></span><ul class="toc-item"><li><span><a href="#Load-the-Dataset-and-Transform" data-toc-modified-id="Load-the-Dataset-and-Transform-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load the Dataset and Transform</a></span></li><li><span><a href="#Build-Association-Rules" data-toc-modified-id="Build-Association-Rules-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Build Association Rules</a></span></li><li><span><a href="#Using-the-Rules" data-toc-modified-id="Using-the-Rules-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Using the Rules</a></span></li></ul></li></ul></div>

 #  Discovering customer attrition patterns 

In this example, we analyze customer attrition data to discover patterns. These will help us dive deeper into those patterns and do root cause analysis of why they are happening. We will use association rules mining algorithm for this purpose.

## Load the Dataset and Transform

We first load the data and view it.


In [3]:
# Import relevant libraries packages
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from apyori import apriori

# Load the prospect dataset
raw_data = pd.read_csv("attrition.csv")

# Preview data
raw_data.head()

,LIFETIME,TYPE,REASON,AGE_GROUP,EMP_STATUS,MARITAL_STATUS,RENEWALS,PROBLEMS,OFFERS
0,1 - 3 M,CANCEL,BETTER DEALS,< 20,STUDENT,SINGLE,0,0 to 5,0 to 2
1,1 - 3 M,CANCEL,BETTER DEALS,< 20,STUDENT,SINGLE,0,0 to 5,0 to 2
2,1Y - 2Y,CANCEL,NOT HAPPY,30 - 50,EMPLOYED,MARRIED,1,10 plus,0 to 2
3,1Y - 2Y,EXPIRY,BETTER DEALS,30 - 50,EMPLOYED,MARRIED,1,0 to 5,2 to 5
4,1Y - 2Y,CANCEL,NOT HAPPY,30 - 50,UNEMPLOYED,SINGLE,1,10 plus,0 to 2


The CSV contains information about each customer who have left the business. It contains attributes like LIFETIME of the customer, How the customer left, reasons, problems and demographics.

For doing association rules mining, the data needs to be in a specific format. Each line should be a transaction with a list of items for that transaction. We will take the CSV file data convert them into values like "name=value" to create this specific data structure

In [5]:
# Data Association Rules Mining
# Convert data to required formatting 
basket_str = ""
for rowNum, row in raw_data.iterrows(): # Use iterrows()
    
    # Break lines
    if (rowNum != 0): # if row number is not the header
        basket_str = basket_str + "\n"
    # Add the rowid as the first column
    basket_str = basket_str + str(rowNum) 
    # Add columns
    for colName, col in row.iteritems():
        basket_str = basket_str + ",\"" + colName + "=" + str(col) +"\""

# Print(basket_str)
basket_file = open("warranty_basket.csv","w")
basket_file.write(basket_str)
basket_file.close()

In [7]:
# Read created file w/o header & preview
basket_data = pd.read_csv("warranty_basket.csv", header = None)
basket_data.head()

,0,1,2,3,4,5,6,7,8,9
0,0,LIFETIME=1 - 3 M,TYPE=CANCEL,REASON=BETTER DEALS,AGE_GROUP=< 20,EMP_STATUS=STUDENT,MARITAL_STATUS=SINGLE,RENEWALS=0,PROBLEMS=0 to 5,OFFERS=0 to 2
1,1,LIFETIME=1 - 3 M,TYPE=CANCEL,REASON=BETTER DEALS,AGE_GROUP=< 20,EMP_STATUS=STUDENT,MARITAL_STATUS=SINGLE,RENEWALS=0,PROBLEMS=0 to 5,OFFERS=0 to 2
2,2,LIFETIME=1Y - 2Y,TYPE=CANCEL,REASON=NOT HAPPY,AGE_GROUP=30 - 50,EMP_STATUS=EMPLOYED,MARITAL_STATUS=MARRIED,RENEWALS=1,PROBLEMS=10 plus,OFFERS=0 to 2
3,3,LIFETIME=1Y - 2Y,TYPE=EXPIRY,REASON=BETTER DEALS,AGE_GROUP=30 - 50,EMP_STATUS=EMPLOYED,MARITAL_STATUS=MARRIED,RENEWALS=1,PROBLEMS=0 to 5,OFFERS=2 to 5
4,4,LIFETIME=1Y - 2Y,TYPE=CANCEL,REASON=NOT HAPPY,AGE_GROUP=30 - 50,EMP_STATUS=UNEMPLOYED,MARITAL_STATUS=SINGLE,RENEWALS=1,PROBLEMS=10 plus,OFFERS=0 to 2


In [13]:
# Drop extra index
filt_data = basket_data.drop(basket_data.columns[[0]], axis = 1)
filt_data.head()

,1,2,3,4,5,6,7,8,9
0,LIFETIME=1 - 3 M,TYPE=CANCEL,REASON=BETTER DEALS,AGE_GROUP=< 20,EMP_STATUS=STUDENT,MARITAL_STATUS=SINGLE,RENEWALS=0,PROBLEMS=0 to 5,OFFERS=0 to 2
1,LIFETIME=1 - 3 M,TYPE=CANCEL,REASON=BETTER DEALS,AGE_GROUP=< 20,EMP_STATUS=STUDENT,MARITAL_STATUS=SINGLE,RENEWALS=0,PROBLEMS=0 to 5,OFFERS=0 to 2
2,LIFETIME=1Y - 2Y,TYPE=CANCEL,REASON=NOT HAPPY,AGE_GROUP=30 - 50,EMP_STATUS=EMPLOYED,MARITAL_STATUS=MARRIED,RENEWALS=1,PROBLEMS=10 plus,OFFERS=0 to 2
3,LIFETIME=1Y - 2Y,TYPE=EXPIRY,REASON=BETTER DEALS,AGE_GROUP=30 - 50,EMP_STATUS=EMPLOYED,MARITAL_STATUS=MARRIED,RENEWALS=1,PROBLEMS=0 to 5,OFFERS=2 to 5
4,LIFETIME=1Y - 2Y,TYPE=CANCEL,REASON=NOT HAPPY,AGE_GROUP=30 - 50,EMP_STATUS=UNEMPLOYED,MARITAL_STATUS=SINGLE,RENEWALS=1,PROBLEMS=10 plus,OFFERS=0 to 2


## Build Association Rules

We now use the apriori algorithm to build association rules. We then extract the results and populate a data frame for future use. The apriori provides the LHS for multiple combinations of the items. We capture the counts along with confidence and lift in this example

In [15]:
# Read back
results = list(apriori(filt_data.to_numpy()))

In [16]:
rulesList = pd.DataFrame(columns = ('LHS', 'RHS', 'COUNT', 'CONFIDENCE','LIFT'))
rowCount = 0

# Convert results into a Data Frame
for row in results:
    for affinity in row[2]:
        rulesList.loc[rowCount] = [ ', '.join(affinity.items_base) ,\
                                    affinity.items_add, \
                                    len(affinity.items_base) ,\
                                    affinity.confidence,\
                                    affinity.lift]
        rowCount +=1

## Using the Rules

We can take a look at the rules by simply doing a head.

In [17]:
rulesList.head()

,LHS,RHS,COUNT,CONFIDENCE,LIFT
0,,(AGE_GROUP=20 - 30),0,0.34,1.0
1,,(AGE_GROUP=30 - 50),0,0.32,1.0
2,,(AGE_GROUP=50PLUS ),0,0.16,1.0
3,,(AGE_GROUP=< 20),0,0.18,1.0
4,,(EMP_STATUS=EMPLOYED),0,0.54,1.0


We can also filter rules where the count of elements is 1 and the confidence is > 70%

In [21]:
rulesList[(rulesList.COUNT <= 1) & (rulesList.CONFIDENCE > 0.7)].head(5).sort_values(by=['LIFT'], ascending=False)

,LHS,RHS,COUNT,CONFIDENCE,LIFT
79,AGE_GROUP=30 - 50,(LIFETIME=1Y - 2Y),1,1.000000,3.125000
80,LIFETIME=1Y - 2Y,(AGE_GROUP=30 - 50),1,1.000000,3.125000
38,LIFETIME=3M to 1Y,(AGE_GROUP=20 - 30),1,1.000000,2.941176
83,MARITAL_STATUS=MARRIED,(AGE_GROUP=30 - 50),1,0.833333,2.604167
70,AGE_GROUP=20 - 30,(TYPE=CANCEL),1,0.941176,1.568627


Looking at the rules, we can easily see some patterns. Customers who have left the business between 3 months and 1 year are always in the age group 20-30. Similarly, customers in age group 20-30 always cancelled the service. These are interesting facts that can be analyzed further by the business.